# My RNN Language Model implementation in Keras

In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
print(tf.__version__)

2.0.0


In [2]:
with open('../datasets/username_dataset.txt', 'r') as file:
    data = file.readlines()
    
data = [example.strip().lower() for example in data]

with open('../datasets/username_dataset.txt', 'r') as file:
    data_str = file.read().lower()

vocab = list(set(data_str))
vocab.sort()

vocab.insert(0, '>') # End token: <end>
vocab.insert(0, '<') # Start token: <start>
vocab.insert(0, '*') # Padding

char_to_index = {ch: i for i, ch in enumerate(vocab)}
index_to_char = {i: ch for i, ch in enumerate(vocab)}

print(char_to_index)

longest_word = max(data, key=len)

print(f"Number of characters: {len(data_str)}")
print(f"Number of unique characters: {len(vocab)}")
print(f"Longest word: {longest_word}")
print(f"Length of longest word: {len(longest_word)}") # Needed information to know what length to pad the usernames to.

{'*': 0, '<': 1, '>': 2, '\n': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, 'a': 14, 'b': 15, 'c': 16, 'd': 17, 'e': 18, 'f': 19, 'g': 20, 'h': 21, 'i': 22, 'j': 23, 'k': 24, 'l': 25, 'm': 26, 'n': 27, 'o': 28, 'p': 29, 'q': 30, 'r': 31, 's': 32, 't': 33, 'u': 34, 'v': 35, 'w': 36, 'x': 37, 'y': 38, 'z': 39}
Number of characters: 401244
Number of unique characters: 40
Longest word: rabbitsreviews
Length of longest word: 14


In [3]:
def print_mapping(mapping):
    print('{')
    for char, _ in zip(mapping, range(20)):
        print('  {:4s}: {:3d},'.format(repr(char), mapping[char]))
    print('  ...\n}')
    
print_mapping(char_to_index)

{
  '*' :   0,
  '<' :   1,
  '>' :   2,
  '\n':   3,
  '0' :   4,
  '1' :   5,
  '2' :   6,
  '3' :   7,
  '4' :   8,
  '5' :   9,
  '6' :  10,
  '7' :  11,
  '8' :  12,
  '9' :  13,
  'a' :  14,
  'b' :  15,
  'c' :  16,
  'd' :  17,
  'e' :  18,
  'f' :  19,
  ...
}


In [4]:
def prepare_dataset(data, username_length):
    """Adds <start> and <end> tokens and pads the usernames."""
    data_X = []
    data_Y = []
    for username in data:
        pad = "*" * (username_length - len(username))
        X = np.array([char_to_index[char] for char in f"<{username}{pad}"]) #.reshape((username_length + 1, 1))
        Y = np.array([char_to_index[char] for char in f"{username}>{pad}"]) #.reshape((username_length + 1, 1))
        data_X.append(X)
        data_Y.append(Y)
    

    return (np.array(data_X), np.array(data_Y))

data = prepare_dataset(data, len(longest_word))

print(data[0].shape)
print(data[1].shape)
dataset = tf.data.Dataset.from_tensor_slices(data)

(54037, 15)
(54037, 15)


In [5]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join([index_to_char[i] for i in input_example.numpy()])))
    print('Target data:', repr(''.join([index_to_char[i] for i in target_example.numpy()])))
    
for input_example, target_example in dataset.take(1):
    print('Input data: ', input_example.numpy())
    print('Target data:', target_example.numpy())

Input data:  '<info**********'
Target data: 'info>**********'
Input data:  [ 1 22 27 19 28  0  0  0  0  0  0  0  0  0  0]
Target data: [22 27 19 28  2  0  0  0  0  0  0  0  0  0  0]


In [6]:
BATCH_SIZE = 128
BUFFER_SIZE = 10000
EPOCHS = 100
VOCAB_SIZE = len(vocab)
RNN_UNITS = 512
EMBEDDING_DIM = 256

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: ((128, 15), (128, 15)), types: (tf.int32, tf.int32)>


In [7]:
def build_model(vocab_size, rnn_units, batch_size, embedding_dim, stateful=False):
    model = tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, batch_input_shape=[batch_size, None]),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.GRU(rnn_units,
                             return_sequences=True,
                             stateful=stateful,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size, activation=tf.nn.softmax)])
    
    return model

In [8]:
model = build_model(
    vocab_size = VOCAB_SIZE,
    rnn_units=RNN_UNITS,
    embedding_dim=EMBEDDING_DIM,
    batch_size=BATCH_SIZE)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (128, None)               0         
_________________________________________________________________
embedding (Embedding)        (128, None, 256)          10240     
_________________________________________________________________
gru (GRU)                    (128, None, 512)          1182720   
_________________________________________________________________
dense (Dense)                (128, None, 40)           20520     
Total params: 1,213,480
Trainable params: 1,213,480
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                               min_delta=0,
                                               patience=5,
                                               verbose=0,
                                               mode='auto',
                                               baseline=None,
                                               restore_best_weights=True)
# MAKES NO SENSE

In [12]:
model.fit(dataset, epochs=EPOCHS, callbacks=[early_stopping])

Epoch 1/100
422/422 [==============================] - 77s 182ms/step - loss: 1.3524
Epoch 2/100
422/422 [==============================] - 71s 167ms/step - loss: 1.1823
Epoch 3/100
422/422 [==============================] - 70s 165ms/step - loss: 1.1423
Epoch 4/100
422/422 [==============================] - 70s 166ms/step - loss: 1.1050
Epoch 5/100
422/422 [==============================] - 72s 171ms/step - loss: 1.0717
Epoch 6/100
422/422 [==============================] - 61s 143ms/step - loss: 1.0428
Epoch 7/100
422/422 [==============================] - 59s 140ms/step - loss: 1.0167
Epoch 8/100
422/422 [==============================] - 60s 141ms/step - loss: 0.9927
Epoch 9/100
422/422 [==============================] - 60s 143ms/step - loss: 0.9714
Epoch 10/100
422/422 [==============================] - 60s 141ms/step - loss: 0.9516
Epoch 11/100
422/422 [==============================] - 60s 142ms/step - loss: 0.9336
Epoch 12/100
422/422 [==============================] - 60s 142

KeyboardInterrupt: 

In [13]:
# model.save_weights('../weights/gru_model_embedding_weights.h5')

In [18]:
model = build_model(VOCAB_SIZE, RNN_UNITS, 1, EMBEDDING_DIM, stateful=True)

In [19]:
model.load_weights('../weights/gru_model_embedding_weights.h5')

In [28]:
def generate_usernames(model, num_generate):
    generated_usernames = []

    for i in range(num_generate):
        model.reset_states()
        input_eval = np.array([char_to_index['<']]).reshape((1, 1))  # We start with the '<start>' token
        generated_username = []
        done = False
        
        while not done:
            predictions = model.predict(input_eval)
            predictions = tf.squeeze(predictions)

            predicted_id = np.random.choice(range(40), p=predictions.numpy())
            # predicted_id = np.argmax(predictions)

            input_eval = np.array([predicted_id]).reshape((1, 1))
            
            done = index_to_char[predicted_id] in ['>', '*']
            
            if not done:
                generated_username.append(index_to_char[predicted_id])
        generated_usernames.append("".join(generated_username))

    return generated_usernames

In [29]:
heatmap = dict()

In [30]:
for i in range(20000):
    generated_usernames = generate_usernames(model, 7)
    
    for username in generated_usernames:
        if heatmap.get(username) is not None:
            heatmap[username] += 1
        else:
            heatmap[username] = 1
    
    print(f'{generated_usernames}: {7 * i}')

['alexh', 'bartura', 'backseat', 'bigben', 'home', 'grapes', 'littlejo']: 0
['hornydog', 'gilman', 'puppie', 'draco', 'petrov', 'bigdaddy', 'whitney']: 7
['scholer', 'pilotb', 'surfer69', 'windoch', 'buddha', 'bigtruch', 'stellio']: 14
['candys', 'currie', 'thegore', 'tonitoni', 'killth', 'hackerbee', 'meteor']: 21
['labs', 'gangel', 'film2', 'blackos', 'longbow', 'daboss', 'ducane']: 28
['greenjherre', 'brians', 'stather', 'brutus', 'bigrick', 'vo5a', 'gautil']: 35
['daveman', 'coreyou', 'ringo1', 'dingking', 'gally', 'rip', 'erot']: 42
['kayfosh', 'felipe', 'penman', 'peterse', 'mwalls', 'hammers', 'zeeba']: 49
['sunny', 'joel', 'barr', 'oman1', 'hell', 'hardy', 'dolemite']: 56
['scal1', 'conal', 'kenny2', 'marvel', 'boo', 'koreysw', 'buttman']: 63
['nathan1', 'gratic1', 'topper', 'stallone', 'madmax', 'seminole', 'minfun']: 70
['deman', 'cobra', 'sach', 'willer', 'toratoro', 'nudi01', 'mercedes']: 77
['termite', 'sacramento', 'baypey', 'brunos', 'bernard', 'jasong', 'kelly']: 84
['n

['laughs', 'bioney', 'tana', 'daniele', 'gorge', 'creamy', 'crappy']: 749
['pummas', 'foxtus', 'doghouse', 'albion', 'castro', 'sender', 'gregory']: 756
['mombo', 'guyguy', 'spartan1', 'perhun', 'stuff', 'gucci', 'fotman']: 763
['cheeks', 'misty', 'dastruth', 'skimpo', 'solohen', 'claris', 'abe']: 770
['hotlady', 'boomer', 'tallis', 'bigmac', 'rayman', 'dystro', 'slipper']: 777
['natalia', 'darkangel', 'diet', 'caveman', 'wbrown', 'roller', 'thalhamm']: 784
['pomme', 'stepping', 'tiffer', 'shegjonn', 'juicy1', 'bernd', 'bunly']: 791
['arrow', 'abcoat', 'bawlet', 'blahblah', 'drifter', 'latition', 'greendog']: 798
['nivek', 'magister', 'jimbo6', 'jason2', 'sandy', 'wildlife', 'saleen']: 805
['chloe', 'horny', 'pokey', 'allrock', 'poontang', 'womens', 'blahim']: 812
['murrah', 'skirpy', 'lugh1776', 'bluto', 'phil', 'trinity', 'cavsecres']: 819
['douggie', 'hummer', 'jamesq', 'strictg', 'oreel', 'buddie', 'mike78']: 826
['rattle', 'doghouse', 'finger', 'marchl', 'jarrech', 'samgster', 'ha

['loberto', 'supa17', 'hardass', 'price', 'mistral', 'josephs', 'milford']: 1491
['bralius', 'tina1', 'boss', 'ramico', 'chigger', 'forward', 'barheed']: 1498
['heather', 'teyose', 'bammer', 'tractor', 'nate', 'simpleman', 'indianl']: 1505
['redstorm', 'francy', 'dimath', 'empire', 'paddle', 'howard', 'checky']: 1512
['sandrine', 'tanto', 'dukelo', 'sierra7', 'quy', 'crotch', 'mademan']: 1519
['barry', 'amy', 'corbett', 'summer', 'richieg', 'jesemas', 'heralder']: 1526
['michael1', 'tyrang', 'tenning', 'cuth', 'france', 'hankey', 'triple']: 1533
['golfptre', 'damond', 'sexbomb', 'stinky', 'slover', 'nimzie', 'dominion']: 1540
['solution', 'olivis', 'broadcast', 'coolie', 'bartolo', 'alvaro', 'ally']: 1547
['bandit2', 'asill', 'wild', 'dondon', 'fomal', 'kevinf', 'hollination']: 1554
['slider', 'sharlene', 'shortee', 'archie', 'battle', 'shultz', 'triplex']: 1561
['andrewd', 'ser3', 'biggy', 'bones', 'capitol', 'raul', 'bethel']: 1568
['geeker', 'sookie', 'caterham', 'hawky', 'cudgle', 

['bgrant', 'seed', 'brancusi', 'sigma', 'hughes', 'frankie', 'reggae']: 2233
['einkauf', 'jakejim', 'pcs', 'starship', 'wetone', 'bowdown', 'beastman']: 2240
['heart', 'miller', 'richardd', 'cfrank', 'trench', 'derdef', 'day']: 2247
['kears', 'larryl', 'hugo', 'camell', 'mthomet', 'office', 'iwant']: 2254
['rbryant', 'luckey', 'simon', 'patrics', 'singing', 'sophia', 'basement']: 2261
['slutty', 'hornet', 'becca', 'sperley', 'daryle', 'urricion', 'penpen']: 2268
['penny', 'scully', 'loveu', 'derwood', 'billyman', 'bigbam', 'bigwood']: 2275
['aprio', 'briguy', 'winger', 'fall', 'pynchon', 'kyle', 'tuaxx2']: 2282
['serino', 'stonecol', 'tara', 'cabowa', 'roody', 'latin1', 'alanm']: 2289
['olek', 'hemi', 'atlast', 'mcbill', 'sarah', 'brenna', 'drifder']: 2296
['hard', 'temper7', 'michaelw', 'lambo1', 'bridgett', 'bryon', 'are']: 2303
['loop', 'sandy', 'canecade', 'custer', 'braxton', 'faswise', 'marilyn']: 2310
['ruhe', 'spriggan', 'gaston', 'skeet', 'squid', 'blindbill', 'russelle']: 231

['bernie', 'cedar', 'jake', 'denali', 'flippy', 'hitman', 'account']: 2982
['mcfadden', 'jomann', 'mentos', 'howard', 'ductiline', 'conino', 'hammer']: 2989
['cheater', 'andie', 'afford', 'john1', 'mete', 'mrdave', 'tablon']: 2996
['provide', 'mister', 'symon', 'abra', 'sammyboy', 'frantic', 'jacky']: 3003
['buyer', 'bigrick', 'admin', 'jeffy', 'crazydog', 'appleman', 'jhill']: 3010
['mont', 'barcas', 'telera', 'james', 'jimmy1', 'jetstorm', 'hugogone']: 3017
['markel', 'sora99', 'stanford', 'drorg', 'pixies', 'sheries', 'halifiia']: 3024
['eagley', 'dolphing', 'bosses', 'colone', 'mill', 'adriano', 'bee']: 3031
['caring', 'ghost69', 'razor1', 'bernard24', 'duncan', 'heddie', 'serpent']: 3038
['boris', 'cdbeach', 'statius', 'advonal', 'sturr1', 'smaster', 'redhear']: 3045
['tarn', 'ozor', 'seraph', 'dling', 'harolo', 'ha8dman', 'heible']: 3052
['dellboy', 'ju', 'fireman', 'gorski', 'stubfs', 'security', 'moose']: 3059
['battle', 'knighth', 'backnot', 'davos', 'erics', 'single', 'filet'

...

['krishna', 'magnum', 'eric', 'pipipip', 'misty', 'burilet', 'ave']: 25088
['topsite', 'paul13', 'lynni', 'funcius', 'sho', 'plat', 'mertho']: 25095
['riddick', 'logos', 'alexes', 'littlered', 'softail', 'markus', 'jimbob']: 25102
['eric2', 'chrise', 'frissy', 'crasher', 'solara', 'format', 'roace']: 25109
['jakey', 'hockey', 'lippy', 'hawkeye', 'tripley', 'dabrass', 'maddog2']: 25116
['thirdman', 'chambers', 'dingdong', 'roger1', 'gregory', 'liberty', 'fermin']: 25123
['jimmyk9', 'radan', 'together', 'david1', 'enter', 'bambam', 'hippie']: 25130
['jezz', 'holly', 'toger', 'webtney', 'cadrand', 'grande', 'bachus']: 25137
['bermex', 'tentation', 'trisko', 'teddy', 'andre', 'deman', 'moody']: 25144
['kat', 'gar', 'warriors', 'jaja', 'oldstorm', 'strife', 'cherish']: 25151
['jacko', 'hill', 'bodger', 'hunger', 'outlaw', 'namo', 'snyder']: 25158
['lbruce', 'puddie', 'lastman', 'sexiest', 'petersen', 'devious', 'theman']: 25165
['miami', 'grainger', 'frankthetank', 'blahblahin', 'blackbird'

['hugo', 'badgers', 'dillweed', 'callum', 'lile', 'para', 'mohammed']: 25823
['briscoe', 'kastle', 'sergia', 'enders', 'distur', 'taters', 'babe']: 25830
['ashusa', 'titan', 'torian', 'salvage', 'bobby1', 'goodgirl', 'davea']: 25837
['trial', 'hammy', 'pricey', 'holy', 'kings', 'meow', 'johnna']: 25844
['sas', 'forbit', 'paola', 'luzer', 'screwer', 'jacky', 'cameleon']: 25851
['hellomoto', 'john83', 'bruman', 'admir', 'bost', 'fischer', 'dilbert']: 25858
['alltime', 'bilbo', 'tiny', 'polad', 'gugu', 'cheeser', 'ade']: 25865
['dyran', 'miamia', 'matth', 'shib', 'hallon1', 'therock', 'colinj']: 25872
['surf', 'robertv', 'buffcrof', 'cherish', 'petrol', 'hottup', 'rickg']: 25879
['luthier', 'almond', 'thebest', 'blastoff', 'kryn', 'highlander', 'slipstone']: 25886
['barrett', 'jeff28', 'brenda1', 'travis7', 'polopony', 'corkett', 'gumpy']: 25893
['shogun', 'sparkman', 'dupett', 'svetasara', 'trumpet', 'anthony', 'robin']: 25900
['lipdog', 'jason5', 'craigius', 'ludovic', 'ringo', 'nimb', 

['allen2', 'anselmo', 'marsace', 'houston', 'groups', 'supadave', 'randolf']: 26558
['yankees', 'robm', 'sally', 'michael3', 'kimber', 'jimmyz', 'gianla']: 26565
['arcana', 'womble', 'hater', 'dean', 'intense', 'topkicc', 'ludovic']: 26572
['shephorn', 'jmiles', 'temp123', 'member', 'barbou', 'godhead', 'justus']: 26579
['andrewm', 'smack', 'data', 'selfg', 'buggsy', 'pervy', 'calicha']: 26586
['foss', 'short', 'holla', 'sturgeon', 'estate', 'chef69', 'creeker']: 26593
['supa', 'inhard', 'martha', 'madmulle', 'ilovegirls', 'tigre', 'laurance']: 26600
['sch2', 'evilone', 'rebwal', 'dogg', 'solge', 'freeman', 'baby']: 26607
['gogot', 'success', 'editer', 'sirphy', 'melmel', 'aave', 'hellcat']: 26614
['jacross', 'shelder', 'thomas35', 'lusty', 'stickit', 'sstacy', 'salami']: 26621
['quickdraw', 'hollyb', 'fender', 'parade', 'trainer', 'jwilson', 'compsoft']: 26628
['susan', 'allie', 'etclub', 'cruncher', 'nemts', 'sams', 'sammy']: 26635
['bthomas', 'kistus', 'shaft', 'promo', 'joester', '

['rawpird', 'deman', 'dandyp', 'jonny', 'nubian', 'file', 'rhillah']: 27293
['break', 'magician', 'dagger', 'matheson', 'diusex', 'guda', 'sliv']: 27300
['abraham', 'sexmania', 'bubba1', 'promo', 'outhouse', 'tripper', 'storig']: 27307
['tlevers', 'horny', 'abc', 'little', 'mariner', 'berner', 'kiri']: 27314
['jann', 'lider', 'dutch', 'orange', 'getting', 'rednek', 'silenther']: 27321
['jakeee', 'ace', 'conner', 'sgriffin', 'borldick', 'toynat', 'manriggy']: 27328
['eds', 'jgraham', 'johnny', 'beachball', 'arinda', 'stuntman', 'trey']: 27335
['shoute', 'jimt', 'garyboy', 'fearless', 'gabriel', 'history', 'jess']: 27342
['danders', 'triplex', 'leolion', 'limer', 'montblas', 'benghem', 'romanp']: 27349
['rambler', 'puppie', 'tigrou', 'estelle', 'jamiea', 'condact', 'tantel']: 27356
['ellock', 'kevin', 'contato', '1god', 'devildog', 'squzzy', 'nerd']: 27363
['lacek', 'oldone', 'reven', 'bob4', 'autgirl', 'luccie', 'bersere']: 27370
['base', 'davidk', 'bigrich', 'judge', 'less', 'starligh'

KeyboardInterrupt: 

In [32]:
print(len(heatmap))
for username, occurence in sorted(heatmap.items(),
                                  key=lambda item: item[1],
                                  reverse=True):
    print(f'{username}: {occurence}')

18630
miller: 41
bigdaddy: 36
bigdick: 24
tema: 23
larry: 21
elvis: 20
buddy: 20
christopher: 19
denny: 18
carlos: 17
simon: 17
therock: 17
hammer: 17
hammy: 17
bigdog: 16
fisher: 16
sally: 16
stephen: 15
bastard: 15
seller: 15
sergio: 15
matthew: 14
hugo: 14
alexander: 13
storm: 13
sly: 13
compton: 13
dale: 13
single: 13
basser: 13
cyber: 13
slicer: 13
olivia: 13
allen: 13
marion: 12
spider: 12
terry: 12
theman: 12
heather: 12
tony: 12
dean: 12
barry: 12
mercury: 12
hamish: 12
jasper: 12
buster: 12
harry: 12
shooter: 12
homer: 12
castor: 12
mikey: 11
gregory: 11
alpha: 11
scorpio: 11
brent: 11
kenny: 11
moondog: 11
timothy: 11
hammie: 11
skyler: 11
bryan: 11
handy: 11
shorty: 11
hammers: 10
skippy: 10
sparky: 10
joe: 10
ericc: 10
erica: 10
triplex: 10
carlos1: 10
teresa: 10
daddy: 10
boss: 10
dominion: 10
candid: 10
charles: 10
slick: 10
iceman: 10
topgun: 10
digger: 10
mister: 10
brown: 10
bobby1: 10
integ: 10
ander: 10
scorpion: 10
conner: 10
roman: 10
kelly: 9
holy: 9
slipper: 9
ta

tripping: 3
chardy: 3
corina: 3
testala: 3
kristin: 3
garymac: 3
frisco: 3
matthieu: 3
skydog: 3
paula1: 3
sarah1: 3
gates: 3
sexylady: 3
indio: 3
pilots: 3
supersex: 3
molson: 3
alexandria: 3
priest: 3
forbes: 3
curious: 3
antares: 3
fozzy: 3
media: 3
beaker: 3
sky1: 3
miken: 3
krazy: 3
ichiro: 3
lesbian: 3
krusty: 3
edmond: 3
mattes: 3
trippy: 3
crappy: 3
solohen: 3
boomer: 3
bluto: 3
mica: 3
sexman: 3
carloso: 3
tuff: 3
vanman: 3
donb: 3
marilyn: 3
marius: 3
schumi: 3
sammyboy: 3
rockets: 3
fore: 3
broken: 3
insertion: 3
rusty: 3
allyn: 3
michell: 3
libby: 3
bigjack: 3
fingers: 3
asterix: 3
davidl: 3
stevel: 3
gilberto: 3
milford: 3
hardman: 3
dimitris: 3
spieler: 3
sucker: 3
masato: 3
ericn: 3
calgary: 3
lostboy: 3
cooter: 3
michael2: 3
racoon: 3
jamesl: 3
semen: 3
shibain: 3
mate: 3
ranma: 3
jonathan: 3
jens: 3
mal: 3
parman: 3
tellus: 3
chico: 3
creeper: 3
leeroy: 3
christop: 3
marisa: 3
allend: 3
timmy: 3
percival: 3
steels: 3
totall: 3
kattie: 3
tennis: 3
perucho: 3
fordman: 3


freddie: 2
boss69: 2
crouse: 2
edison: 2
luis: 2
kuka: 2
recopi: 2
jesser: 2
mark1: 2
spudley: 2
sporting: 2
granty: 2
bondage: 2
ronald: 2
klee: 2
halton: 2
stutt: 2
richard1: 2
marine1: 2
miker: 2
toma: 2
blackpanther: 2
manderson: 2
fredraby: 2
boobed: 2
tom21: 2
kato: 2
damage: 2
maniax: 2
brianc: 2
sandee: 2
chris20: 2
dgreen: 2
seanmc: 2
primate: 2
antin: 2
jameshollands: 2
homes: 2
abbot: 2
hilton: 2
russells: 2
docd: 2
spycam: 2
liners: 2
area69: 2
matth: 2
ami: 2
arden: 2
schneider: 2
dufour: 2
sun: 2
karls: 2
piglet: 2
stever: 2
israel: 2
glynn: 2
jpitts: 2
gordo: 2
jeanna: 2
aaron8: 2
dantemp: 2
dgrove: 2
internet: 2
assemb: 2
stain: 2
marco1: 2
antonio1: 2
bineau: 2
shasta: 2
hartmann: 2
dumbass: 2
jamesf: 2
loft: 2
moemoe: 2
tolik: 2
grumpy: 2
andrew2: 2
rodriguez: 2
computer: 2
account: 2
mentos: 2
mrdave: 2
provide: 2
frantic: 2
jacky: 2
admin: 2
jhill: 2
bosses: 2
colone: 2
harolo: 2
dellboy: 2
ju: 2
moose: 2
tommylee: 2
manco: 2
faber: 2
michaelm: 2
nickb: 2
mario: 2
t

marwin: 2
nick: 2
bismark: 2
tne: 2
noneya: 2
heathrow: 2
shuffle: 2
scoots: 2
bates: 2
awful: 2
susanc: 2
come: 2
alessa: 2
bungee: 2
pooper: 2
songoku: 2
matthe: 2
robo: 2
nobby: 2
icaro: 2
holyshit: 2
racker: 2
tonyyy: 2
aron: 2
gattor: 2
grande: 2
barbi: 2
geeman: 2
babe: 2
talisker: 2
camele: 2
griffin: 2
liza: 2
basco: 2
nitrous: 2
bluejay: 2
peperone: 2
veritas: 2
michael5: 2
oral: 2
superbob: 2
sports: 2
payton: 2
slimpy: 2
seagull: 2
templar1: 2
supa: 2
toothless: 2
fantasy: 2
form: 2
stutter: 2
banana: 2
carmine: 2
interacial: 2
basto: 2
rodger: 2
less: 2
smithers: 2
crash1: 2
diggs: 2
sph: 2
gustav: 2
nitram: 2
rene: 2
oneton: 2
cycle: 2
tinytim: 2
hlarry: 2
audio: 2
arnoud: 2
driver: 2
skins: 2
pettit: 2
chall: 2
cris: 2
luth: 2
mala: 2
keith: 2
regnar: 2
randolph: 2
mojo: 2
constance: 2
kale: 2
signals: 2
buster1: 2
darina: 2
webmaster: 2
bigj: 2
buckman: 2
hahaha: 2
dakota: 2
sunlover: 2
firefly: 2
mikeylikes: 2
sub: 2
davec: 2
loon: 2
cwilliam: 2
mrbill: 2
papajoe: 2
bre

eatme: 2
nesta: 2
sharpy: 2
ryans: 2
senate: 2
goober: 2
silsky: 2
over: 2
magnum: 2
richardc: 2
daves: 2
dland: 2
jorge: 2
kira: 2
bathing: 2
gastro: 2
haras: 2
mmonika: 2
kram: 2
corpse: 2
sexkitten: 2
hunt: 2
belial: 2
thebest: 2
cyrill: 2
aryan: 2
bloody: 2
wyver: 2
giulio: 2
havoc: 2
alkamo: 2
way: 2
nola: 2
carper: 2
charlest: 2
rogers: 2
conny: 2
sweet: 2
cordell: 2
internal: 2
meyer: 2
nicky: 2
chefman: 2
jazzy: 2
drdemara: 2
arnaud: 2
lovem: 2
beavis: 2
slap: 2
tandy: 2
gaysex: 2
hironori: 2
fout: 2
hottie: 2
yuyu: 2
pistol: 2
gmn: 2
sluppy: 2
glenn: 2
fatback: 2
jericho: 2
nomad: 2
macker: 2
emil: 2
aland: 2
talk: 2
simcic: 2
marky: 2
all: 2
tower: 2
buzz: 2
seco: 2
dogman: 2
thomsen: 2
marsel: 2
blotter: 2
highroller: 2
dunhill: 2
gifted: 2
sleeper: 2
thrice: 2
downey: 2
parrott: 2
gator: 2
msimon: 2
sorden: 2
mercy: 2
cliffy: 2
renrew: 2
snoops: 2
andynato: 2
bird: 2
dnsadm: 2
katu: 2
soupy: 2
kaneda: 2
sasa: 2
thelma: 2
gidget: 2
sani: 2
kingston: 2
brian7: 2
buddog: 2
sho

...

calo: 1
shootwock: 1
belom: 1
shithead: 1
sodapop: 1
mariea: 1
mcarter: 1
terrano: 1
ferarri: 1
iraq: 1
myyep: 1
stiff: 1
sunson: 1
thumper7: 1
jholl: 1
emother: 1
leggss: 1
muppie: 1
ralf: 1
columbia: 1
davidoff: 1
daniel1: 1
fgfg: 1
alanka: 1
lambette: 1
blahrd: 1
dmills: 1
richardr: 1
simgi: 1
malya: 1
johnnyx: 1
doof: 1
millenium: 1
festus: 1
packed: 1
mirro: 1
denver: 1
mcball: 1
maks: 1
excter: 1
thirpy: 1
sports1: 1
jiang: 1
0ner: 1
subdom: 1
corleys: 1
myr: 1
mrbob: 1
lovers: 1
kazu: 1
ericvanr: 1
seafood: 1
schide: 1
scottigh: 1
roij: 1
scarab: 1
bumpman: 1
hollybear: 1
marcelle: 1
mil: 1
roborr: 1
kissam: 1
tenford: 1
boothe: 1
matthias: 1
remaorr: 1
bearpe: 1
ferguson: 1
rust: 1
gogw: 1
rapala: 1
eddieo: 1
moosey: 1
mitch: 1
manolo: 1
sedatho: 1
thekidduy: 1
michae: 1
bobby5: 1
blakem: 1
hotrosse: 1
opril: 1
nellie: 1
darky: 1
funkman: 1
denman85: 1
carrie: 1
snag: 1
telguy: 1
norcal: 1
daredevil: 1
spih: 1
larryc: 1
tomator: 1
charlied: 1
glacier: 1
holbride: 1
norwegi: 1
s

sveta: 1
procford: 1
tetran: 1
closhed: 1
razz: 1
dresd: 1
malacria: 1
spaced: 1
octopul: 1
bunnis: 1
haggard: 1
daima: 1
jerree: 1
daly: 1
stass: 1
longer: 1
soa: 1
archange: 1
sisson: 1
fynica: 1
gmoney: 1
milflefi: 1
colles: 1
gangsame: 1
lump: 1
pershing: 1
littlehole: 1
cooldewd: 1
johnmish: 1
hose: 1
rother: 1
flicky: 1
cazzone: 1
superve: 1
james88: 1
seductive: 1
shirber: 1
tomb: 1
hokies: 1
thomas18: 1
thesimp: 1
merod: 1
smalls: 1
lion1: 1
versalea: 1
jamesjones: 1
scootie: 1
jeal: 1
thera: 1
fred87texas: 1
superdde: 1
meridy: 1
momser: 1
zoom: 1
guaut: 1
hot1: 1
tduve: 1
ahsha: 1
givex: 1
katzie: 1
layne: 1
aqua: 1
bigvols: 1
samm: 1
irish: 1
alansis: 1
bmy1: 1
rifle: 1
eleina: 1
trauen: 1
stix: 1
tigereye: 1
cthrip: 1
fumruv: 1
sexyivons: 1
macuser: 1
gazebas: 1
editors: 1
lampe: 1
brad: 1
godo12: 1
blmberr: 1
tight: 1
littledick: 1
banane: 1
alpha06: 1
gibby: 1
takarine: 1
bigmel: 1
fadi: 1
caster: 1
fist: 1
nomolos: 1
bunghole: 1
carl88: 1
congner: 1
bradi: 1
faceman: 1
s

twdog: 1
semret: 1
celtic: 1
bastien: 1
turn123: 1
nooner: 1
felixther: 1
discraft: 1
caino: 1
pounder: 1
ricopu: 1
turbon: 1
wipeou: 1
timegold: 1
tenins: 1
mike51: 1
microld: 1
same: 1
dhello: 1
bluee: 1
malc: 1
holmes: 1
adonis: 1
shaq: 1
argentup: 1
rodler: 1
gunm: 1
jason29: 1
kevinh: 1
gabbya: 1
shroobin: 1
getrae: 1
thomas8: 1
ottie: 1
raikins: 1
thin: 1
stopity: 1
fred1: 1
geor: 1
jack4: 1
mitsu: 1
bucher: 1
marco: 1
strong: 1
rockow: 1
finagle: 1
simmon: 1
aleyanke: 1
leena: 1
cowboys: 1
ssmith: 1
douglus: 1
ellwood: 1
tigro: 1
andy76: 1
humani: 1
robert2: 1
krakatime: 1
sharad: 1
marktit: 1
doorst: 1
kautman: 1
fishhead: 1
crossfire: 1
uriston: 1
jonask: 1
good1: 1
notboy: 1
tile: 1
renault: 1
bugg: 1
peterin: 1
silverfox: 1
terre: 1
gresh: 1
dodged: 1
pvs: 1
jdogg: 1
planeme: 1
jonsey: 1
sheller: 1
supp: 1
red: 1
advance: 1
bobman: 1
hugoiuman: 1
hrjames: 1
pingpong: 1
slarf: 1
ttrapier: 1
halifax: 1
audios: 1
tdogg: 1
safer: 1
lurch: 1
bryson: 1
farhad: 1
rimj: 1
hyundai: 1

jimmmm: 1
jejeje: 1
joesmith: 1
pughea: 1
dale03: 1
goleal: 1
lekker: 1
crybaby: 1
ships: 1
ccthvis: 1
tilly: 1
dogma: 1
clouds: 1
pennywis: 1
saik: 1
roadrash: 1
exersame: 1
jone: 1
stones1: 1
kensey: 1
iamhorse: 1
serggal: 1
etecter: 1
timie: 1
alex69: 1
kozlov: 1
phoeewill: 1
mark123: 1
theresa: 1
dottrot: 1
maskun: 1
nyoh: 1
cowcow: 1
akram: 1
traiser: 1
goya: 1
rondog: 1
total: 1
matrix: 1
doggys: 1
bart83: 1
alefia: 1
ribded: 1
dholland: 1
mementos: 1
sirene: 1
samprias: 1
johndery: 1
randys: 1
cram: 1
tuktuk: 1
hiroshi: 1
martell: 1
eastwest: 1
flatron: 1
antelos: 1
alcat: 1
brenta: 1
penrose: 1
gregdry: 1
model: 1
cree: 1
magicone: 1
goda: 1
face07: 1
desmith: 1
mercer: 1
parkover: 1
hilticio: 1
dereke: 1
beksard: 1
macron: 1
mintim: 1
gerardo: 1
stango: 1
linling: 1
havies: 1
colux: 1
percus: 1
foxtrik: 1
andi: 1
ninebad: 1
ecilope: 1
herolang: 1
cristopher: 1
jayser: 1
rimo: 1
mingo: 1
shelba: 1
heimer: 1
eldar: 1
mollie: 1
salida: 1
djmal: 1
superboy: 1
alcoot: 1
sexson: 1
a

infeman: 1
dopey: 1
cheryl: 1
lakery: 1
morend: 1
suckers: 1
princess: 1
spawns: 1
bigd7: 1
lawry: 1
rrolley: 1
jtravis: 1
hammam: 1
mikedr: 1
dolitt: 1
broncy: 1
areamed: 1
endird: 1
pillock: 1
mikedman: 1
amorrison: 1
pauls: 1
cey: 1
bocs: 1
dave65: 1
panda1: 1
steuth: 1
diary: 1
nehnad: 1
fabiolaa: 1
shyone: 1
herding: 1
cinque: 1
spicard: 1
jwilliam: 1
sladex: 1
crimd: 1
kuke: 1
scooby: 1
bigtrick: 1
hillside: 1
dorsey: 1
lites: 1
jude: 1
skilley: 1
suneital: 1
malo: 1
tristam: 1
beccas: 1
holyway: 1
balloo: 1
evelsen: 1
bozea: 1
cosmos: 1
amethyst: 1
danielg: 1
milano: 1
ballman: 1
aviavi: 1
bluemonkey: 1
slidel: 1
providia: 1
allen2: 1
anselmo: 1
marsace: 1
robm: 1
jimmyz: 1
gianla: 1
arcana: 1
womble: 1
hater: 1
topkicc: 1
shephorn: 1
jmiles: 1
barbou: 1
godhead: 1
selfg: 1
buggsy: 1
calicha: 1
foss: 1
sturgeon: 1
chef69: 1
inhard: 1
madmulle: 1
ilovegirls: 1
laurance: 1
sch2: 1
rebwal: 1
solge: 1
gogot: 1
editer: 1
sirphy: 1
melmel: 1
aave: 1
hellcat: 1
jacross: 1
shelder: 1
st